## XZ-Ordering: A Space-Filling Curve for Objects with Spatial Extension

#### **研究目标：** 如何将空间信息映射到关系型数据库，将多维信息变换为一维数据进行存储管理

### MBR R树索引：
R树中存放的数据并不是原始数据，而是这些数据的最小边界矩形（MBR）
![](img/Rtree.png)
R树平均查询复杂度为O(Log(N))，N为多边形个数

## 空间/语义 信息分开存储：
无法兼顾：
如果一方的更新操作失败，另外一个数据库的更新也必须取消，不能保证数据的物理和逻辑独立性。
## 对象-关系型数据库：
无法提供基于块的索引，大部分的数据库不支持这样的操作。

## 解决方案：
将空间信息进行映射，通过关系型数据库存储。  
虽然无法保证空间信息的绝对精确，但在限制的区域内，每个映射保证空间中的相邻关系在一维表示下的邻近关系。

### 空间填充曲线

### Z-curve  
![Z curve](img/delam.png)  
具有自相似性，但图形中有长边（长距离的连接）。

### Hilbert Curve
![Hilbert curve](img/k-hilbert.png)  

### Peano Curve  
![peano curve](img/peano-grid.png)  

###  数据库中 Z-Order 存储
![Z-Order](img/ZOrder.png)

按照递归进行分解提升分辨率，  
每一次划分为4个子象限  
#### 指代：
|符号|表示|
|:---:|:---:|
|**g**| 递归迭代的固定次数，即分辨率级别|  
|**l**| 一个象限序列的长度|  
|**element**| 每一个象限序列都是一个元素|  
|**cell**| 每一个最小分解（长度为g的象限序列）称为细胞|  
|**prefix**| 如果一个元素 a 包含在另外一个元素 b 内部，则 b 是 a 的前缀|  

1. 象限序列具有 **字典序** 属性  
2. 象限序列越长，表示的区域越小  
3. 在点数据库中，保存的单位是细胞  

### 查询过程：
**查询窗口**  
查询窗口内分为4个象限，对每个象限进行相交性计算  
1. 如果象限与窗口无相交 -> 跳过  
2. 如果象限完全在窗口内部 -> 查询数据库中以该象限为前缀的所有子象限  
3. 如果象限与窗口相交但不完全在窗口内部 -> 逐步进行递归分解，找到最小分解（Basic resolution）  

### 单值逼近  
![](img/onevalueapp.png)

目标物体由最小的元素进行逼近  
流程：  
1. 将当前区域分解为四个子象限  
2. 如果某一个象限与目标相交，则对该象限进行递归分解  
3. 如果多个象限与目标相交，则停止，使用该序列作为排序的键  

这种方法的  
**优点**是每一个目标物体都被一个单独的序列表示，为不是一系列的序列  
**缺点**是每一个目标物体的逼近是<u>长度不同</u>的序列，每一个key都是以可变长度的字符串字典序列进行存储，没有数字类型效率高；  
对于目标物体的表示性差，如果目标物体与轴相交，则必须用空序列（全范围序列）进行表示。

### 优化冗余
*size-bound*  
*error-bound*  


### 代替技术
空间填充曲线

### A Space-Filling Curve for Spatially Extended Objects

1. 相同层级的元素有50%的重叠率（可以避免/降低逼近误差）  
2. 通过编码技术，以变长序列将象限映射到整数域，同时保留空间信息  
3. 查询过程中间隙生成算法  

#### 元素重叠

XZ-Ordering 中每个元素的**左下角**对应与 Z-Ordering 的元素  
每个元素由 左下 -> 右上 进行扩展，  
**s** 表示左下的象限序列，  
**|s|** 表示序列的长度，  
每一个元素的 宽w 和 高h 是 0.5<sup>|s|-1</sup>

![](img/Z-vs-XZ.png)

|Order|w/h|value|
|:---:|:---:|:---:|
|Z-[000]|0.125|0.5<sup>3</sup>|  
|XZ-[000]|0.25|0.5<sup>3-1</sup>|

这里有一个问题是最右边一列和全局右上角如何填充：  
没有位置了不填充，空间相邻（上下左右）的元素仍有 50% 的部分重叠

序列范围：
![](img/object.png)  
000  
001  
002  
003  
010  
012  
2 <= |s| = 3 < 4 


#### 象限序列编码

数字的不等序 对应于 象限序列的字典顺序

![](img/element.jpg)

**l** = <00> -> |2|   
**g** = <0000> -> |4| 

### Cells:  
N<sub>cells</sub>(l) = 4<sup>4-2</sup>=16
### Elements:  
N<sub>elements</sub>(l) = (4<sup>4-2+1</sup>-1)/3=21  
4x4 = 1  
2x2 = 4  
1x1 = 16  

#### 象限序列编码

 s = <q0 q1 ... qi ... ql-1>

C(s) = $\sum_{0\leq i < l}q_i· \frac{4^{g-i}-1}{3}+1$

计算g=4之前的所有序列：

In [1]:
q0 = q1 = q2 = q3 = '0'

In [2]:
def calculate_value2():
    qq0 = int(q0)
    qq1 = int(q1)
    while(qq0 < 4):
        value = 5*qq0 + qq1 + 2
        print("<{}{}> - {}".format(qq0, qq1, value))
        qq1 = qq1 + 1
        if qq1 == 4:
            print()
            qq0 = qq0 + 1
            qq1 = 0

In [9]:
def calculate_value3():
    qq0 = int(q0)
    qq1 = int(q1)
    qq2 = int(q2)
    while(qq0 < 4):
        value = 21*qq0 + 5*qq1 + qq2 + 3
        print("<{}{}{}> - {}".format(qq0, qq1, qq2, value))
        qq2 = qq2 + 1
        if qq2 == 4:
            qq1 = qq1 + 1
            qq2 = 0
            if qq1 == 4:
                qq0 = qq0 + 1
                qq1 = 0
                print()

In [11]:
def calculate_value4():
    qq0 = int(q0)
    qq1 = int(q1)
    qq2 = int(q2)
    qq3 = int(q3)
    while(qq0 < 4):
        value = 85*qq0 + 21*qq1 + 5*qq2 + qq3 + 4
        print("<{}{}{}{}> - {}".format(qq0, qq1, qq2, qq3, value))
        qq3 = qq3 + 1
        if qq3 == 4:
            qq2 = qq2 + 1
            qq3 = 0
            if qq2 == 4:
                qq1 = qq1 + 1
                qq2 = 0
                print()
                if qq1 == 4:
                    qq0 = qq0 + 1
                    qq1 = 0
                print()

In [4]:
s = q0+q2
print(s)

00


<0> = 1  
<1> = 2  
<2> = 3  
<3> = 4  

In [5]:
calculate_value2()

<00> - 2
<01> - 3
<02> - 4
<03> - 5

<10> - 7
<11> - 8
<12> - 9
<13> - 10

<20> - 12
<21> - 13
<22> - 14
<23> - 15

<30> - 17
<31> - 18
<32> - 19
<33> - 20



In [8]:
calculate_value3()

<000> - 3
<001> - 4
<002> - 5
<003> - 6
<010> - 8
<011> - 9
<012> - 10
<013> - 11
<020> - 13
<021> - 14
<022> - 15
<023> - 16
<030> - 18
<031> - 19
<032> - 20
<033> - 21

<100> - 24
<101> - 25
<102> - 26
<103> - 27
<110> - 29
<111> - 30
<112> - 31
<113> - 32
<120> - 34
<121> - 35
<122> - 36
<123> - 37
<130> - 39
<131> - 40
<132> - 41
<133> - 42

<200> - 45
<201> - 46
<202> - 47
<203> - 48
<210> - 50
<211> - 51
<212> - 52
<213> - 53
<220> - 55
<221> - 56
<222> - 57
<223> - 58
<230> - 60
<231> - 61
<232> - 62
<233> - 63

<300> - 66
<301> - 67
<302> - 68
<303> - 69
<310> - 71
<311> - 72
<312> - 73
<313> - 74
<320> - 76
<321> - 77
<322> - 78
<323> - 79
<330> - 81
<331> - 82
<332> - 83
<333> - 84



In [12]:
calculate_value4()

<0000> - 4
<0001> - 5
<0002> - 6
<0003> - 7
<0010> - 9
<0011> - 10
<0012> - 11
<0013> - 12
<0020> - 14
<0021> - 15
<0022> - 16
<0023> - 17
<0030> - 19
<0031> - 20
<0032> - 21
<0033> - 22


<0100> - 25
<0101> - 26
<0102> - 27
<0103> - 28
<0110> - 30
<0111> - 31
<0112> - 32
<0113> - 33
<0120> - 35
<0121> - 36
<0122> - 37
<0123> - 38
<0130> - 40
<0131> - 41
<0132> - 42
<0133> - 43


<0200> - 46
<0201> - 47
<0202> - 48
<0203> - 49
<0210> - 51
<0211> - 52
<0212> - 53
<0213> - 54
<0220> - 56
<0221> - 57
<0222> - 58
<0223> - 59
<0230> - 61
<0231> - 62
<0232> - 63
<0233> - 64


<0300> - 67
<0301> - 68
<0302> - 69
<0303> - 70
<0310> - 72
<0311> - 73
<0312> - 74
<0313> - 75
<0320> - 77
<0321> - 78
<0322> - 79
<0323> - 80
<0330> - 82
<0331> - 83
<0332> - 84
<0333> - 85


<1000> - 89
<1001> - 90
<1002> - 91
<1003> - 92
<1010> - 94
<1011> - 95
<1012> - 96
<1013> - 97
<1020> - 99
<1021> - 100
<1022> - 101
<1023> - 102
<1030> - 104
<1031> - 105
<1032> - 106
<1033> - 107


<1100> - 110
<1101> - 111
<1